# Use the notebook at the Colab.


In [ ]:
%tensorflow_version 1.x

In [ ]:
!pip install pendulum -q

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
!git clone --recursive https://github.com/roymondliao/albert-japanese.git

In [ ]:
!git clone https://github.com/brightmart/albert_zh.git

## Auth GCP account

In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = "<your gcp project id>""
!gcloud config set project {project_id}


In [ ]:
MAX_SEQ_LEN = 512
INPUT_DATA_GCS = "<input data source from gcs>"

In [ ]:
TARGET_DIRS = [
  'AA',
  'AB',
  'AC',
  'AD',
  'AE',
  'AF',
  'AG',
  'AH',
  'AI',
  'AJ',
  'AK',
  'AL',
  'AM',
  'AN',
  'AO',
  'AP',
  'AQ',
  'AR',
  'AS',
  'AT',
  'AU',
  'AV',
  'AW',
  'AX',
  'AY',
  'AZ',
  'BA',
  'BB',
  'BC',
]

In [ ]:
INPUT_FILE = ','.join(['{}/all-maxseq{}_{}.tfrecord'.format(INPUT_DATA_GCS, MAX_SEQ_LEN, elem) for elem in TARGET_DIRS])
INPUT_FILE

## Check TPU devices

In [ ]:
import datetime
import time
import json
import os
import pprint
import random
import string
import pendulum
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
  

In [ ]:
run_id = 'checkpoint_{}'.format(pendulum.now("Asia/Taipei").format("YYYYMMDDHHMMSS"))
run_id

In [ ]:
OUTPUT_GCS = "<output destination>" + '/' + run_id
EXPORT_GCS = "<save model path>"
start_time = time.time()

In [ ]:
!python3 ./albert_zh/run_pretraining_google.py \
--input_file={INPUT_FILE} \
--eval_batch_size=256 \
--output_dir={OUTPUT_GCS} \
--do_train=True \
--do_eval=True \
--albert_config_file=./albert-japanese/models_config/albert_tiny_jp_v2.json  \
--export_dir={EXPORT_GCS} \
--train_batch_size=512 \
--max_seq_length=512 \
--max_predictions_per_seq=20 \
--num_train_steps=250000 \
--num_warmup_steps=25000 \
--learning_rate=0.00176 \
--save_checkpoints_steps=5000 \
--use_tpu=True \
--tpu_name={TPU_ADDRESS} \
--num_tpu_cores=8


In [ ]:
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time / 60 / 60}")


In [ ]:
import os
from tensorflow.python.profiler import profiler_client

tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466')
print(profiler_client.monitor(tpu_profile_service_address, 100, 2))